In [26]:
import os

%cd C:\Users\mathy\Documents\SI5\PER\models

# Cloner YOLOv5 s'il n'existe pas déjà
if not os.path.exists("yolov5"):
    !git clone https://github.com/ultralytics/yolov5

%cd yolov5

#!pip install -r requirements.txt onnx onnxruntime onnx_tf tensorflow


C:\Users\mathy\Documents\SI5\PER\models
C:\Users\mathy\Documents\SI5\PER\models\yolov5
C:\Users\mathy\Documents\SI5\PER\models\yolov5


Cloning into 'yolov5'...


In [28]:
import torch
import os

print("Torch CUDA available:", torch.cuda.is_available())

Torch CUDA available: True


In [29]:
import os
import glob
import random
import shutil

SOURCE_DIR = r"C:\Users\mathy\Documents\SI5\PER\thermal_coco\augmented"
SOURCE_LABELS = r"C:\Users\mathy\Documents\SI5\PER\mini_coco\labels"
TARGET_DIR = "thermal_yolo"

print(f"Source exists: {os.path.exists(SOURCE_DIR)}")
print(f"Labels exist: {os.path.exists(SOURCE_LABELS)}")

os.makedirs(TARGET_DIR, exist_ok=True)

# YOLO folders
folders = ["images/train", "images/val", "labels/train", "labels/val"]
for f in folders:
    os.makedirs(os.path.join(TARGET_DIR, f), exist_ok=True)

# Récupérer toutes les images
images = glob.glob(os.path.join(SOURCE_DIR, "*.jpg"))
print(f"Images trouvées : {len(images)}")

if len(images) == 0:
    print("ERREUR : Aucune image trouvée!")
else:
    # Train/val split
    train_ratio = 0.8
    random.shuffle(images)
    split = int(len(images) * train_ratio)
    
    train_imgs = images[:split]
    val_imgs = images[split:]
    
    print(f"Train: {len(train_imgs)}, Val: {len(val_imgs)}")
    
    # Copier train
    for img_path in train_imgs:
        img_name = os.path.basename(img_path)
        shutil.copy(img_path, os.path.join(TARGET_DIR, "images/train", img_name))
        
        # Récupérer le label original (sans _aug)
        base_name = img_name.split('_aug')[0] if '_aug' in img_name else img_name.replace('.jpg', '')
        label_src = os.path.join(SOURCE_LABELS, base_name + '.txt')
        label_dst = os.path.join(TARGET_DIR, "labels/train", img_name.replace('.jpg', '.txt'))
        
        if os.path.exists(label_src):
            shutil.copy(label_src, label_dst)
    
    # Copier val
    for img_path in val_imgs:
        img_name = os.path.basename(img_path)
        shutil.copy(img_path, os.path.join(TARGET_DIR, "images/val", img_name))
        
        base_name = img_name.split('_aug')[0] if '_aug' in img_name else img_name.replace('.jpg', '')
        label_src = os.path.join(SOURCE_LABELS, base_name + '.txt')
        label_dst = os.path.join(TARGET_DIR, "labels/val", img_name.replace('.jpg', '.txt'))
        
        if os.path.exists(label_src):
            shutil.copy(label_src, label_dst)
    
    print("Dataset YOLO créé")


Source exists: True
Labels exist: True
Images trouvées : 2500
Train: 2000, Val: 500
Dataset YOLO créé
Dataset YOLO créé


In [30]:
# Classes animales uniquement (celles utilisées dans download_mini_coco_animals.py)
ANIMAL_CLASSES = {
    14: "bird",
    15: "cat",
    16: "dog",
    17: "horse",
    18: "sheep",
    19: "cow",
    20: "elephant",
    21: "bear",
    22: "zebra",
    23: "giraffe",
}

classes = list(ANIMAL_CLASSES.values())
print(f"Classes animales : {classes}")

yaml_content = f"""
train: {TARGET_DIR}/images/train
val: {TARGET_DIR}/images/val

nc: {len(classes)}
names: {classes}
"""

with open("data.yaml", "w") as f:
    f.write(yaml_content)

print("data.yaml généré !")


Classes animales : ['bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe']
data.yaml généré !


In [ ]:
!python train.py --img 256 --batch 16 --epochs 50 --weights yolov5n.pt --data data.yaml

In [6]:
!python export.py --weights runs/train/thermal_yolov5n_256x192/weights/best.pt --include onnx --imgsz 256

export: data=C:\Users\abdel\Desktop\SI5\S2\PER\test_conversion_rgb_th\yolov5\data\coco128.yaml, weights=['runs/train/thermal_yolov5n_256x192/weights/best.pt'], imgsz=[256], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, per_tensor=False, dynamic=False, cache=, simplify=False, mlmodel=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
YOLOv5  v7.0-448-gdeec5e45 Python-3.13.3 torch-2.9.1+cpu CPU

Traceback (most recent call last):
  File "c:\Users\abdel\Desktop\SI5\S2\PER\test_conversion_rgb_th\yolov5\export.py", line 1525, in <module>
    main(opt)
    ~~~~^^^^^
  File "c:\Users\abdel\Desktop\SI5\S2\PER\test_conversion_rgb_th\yolov5\export.py", line 1520, in main
    run(**vars(opt))
    ~~~^^^^^^^^^^^^^
  File "c:\Users\abdel\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\utils\_contextlib.py", line 120, in decorate

In [7]:
from onnx_tf.backend import prepare
import onnx

model_onnx = onnx.load("best.onnx")
tf_rep = prepare(model_onnx)
tf_rep.export_graph("saved_model")

ImportError: cannot import name 'mapping' from 'onnx' (c:\Users\abdel\AppData\Local\Programs\Python\Python313\Lib\site-packages\onnx\__init__.py)

In [ ]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model("saved_model")
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # INT8 quantization
tflite_model = converter.convert()

with open("model_int8.tflite", "wb") as f:
    f.write(tflite_model)

print("TFLite model saved as model_int8.tflite!")

In [ ]:
import numpy as np
from PIL import Image
import cv2

interpreter = tf.lite.Interpreter(model_path="model_int8.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

def infer(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (256,192))
    img = img[..., ::-1]  # BGR → RGB
    img = img / 255.0
    img = np.expand_dims(img.astype(np.float32), axis=0)

    interpreter.set_tensor(input_details[0]['index'], img)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])

    print("Output shape:", output.shape)
    return output

# EXEMPLE
# infer("../thermal_dataset/images/val/test.jpg")
